# SANDAG Populationsim QC

In [1]:
import pandas as pd
import numpy as np
import pyodbc
import glob
import copy

# Data Preparation

### Download Populationsim data

In [2]:
synthetic_households = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-72 SANDPOPSIM Output QC/Population Sim Outputs/synthetic_households.csv')
synthetic_persons = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-72 SANDPOPSIM Output QC/Population Sim Outputs/synthetic_persons.csv')

### Download Crosswalk Data

In [3]:
# Download Data
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=SQL2014B8.sandag.org;'
                      'Database=GeoDepot;'
                      'Trusted_Connection=yes;')

query = '''SELECT
      [MGRA]
	  ,cpa_data.NAME AS CPA
	  ,jur_data.Name AS Jurisdiction
      ,[LUZ]
  FROM [GeoDepot].[gis].[MGRA15] as mgra_base
  LEFT JOIN [GeoDepot].[gis].[CITIES] AS jur_data
  ON mgra_base.City = jur_data.City
  LEFT JOIN [GeoDepot].[gis].[CITYCPA] AS cpa_data
  ON mgra_base.CPA = cpa_data.CPA'''

crosswalk = pd.read_sql_query(query,conn)

In [4]:
# Appending to synthetic data
synthetic_households= synthetic_households.merge(crosswalk, left_on='mgra', right_on='MGRA', how='left')
synthetic_persons = synthetic_persons.merge(crosswalk, left_on='mgra', right_on='MGRA', how='left')

In [5]:
synthetic_persons['Region'] = 'San Diego'
synthetic_households['Region'] = 'San Diego'

### Georgaphy Groupings

In [6]:
# Synthetic Population (SP)
sh_mgra = pd.DataFrame(synthetic_households['mgra'].value_counts()).rename(columns={'mgra':'Total'})
sh_mgra.index.name = 'mgra'

sh_tract = pd.DataFrame(synthetic_households['tract'].value_counts()).rename(columns={'tract':'Total'})
sh_tract.index.name = 'tract'

sh_CPA = pd.DataFrame(synthetic_households['CPA'].value_counts()).rename(columns={'CPA':'Total'})
sh_CPA.index.name = 'CPA'

sh_jurisdiction = pd.DataFrame(synthetic_households['Jurisdiction'].value_counts()).rename(columns={'Jurisdiction':'Total'})
sh_jurisdiction.index.name = 'Jurisdiction'

sh_LUZ = pd.DataFrame(synthetic_households['LUZ'].value_counts()).rename(columns={'LUZ':'Total'})
sh_LUZ.index.name = 'LUZ'

sh_Region = pd.DataFrame(synthetic_households['Region'].value_counts()).rename(columns={'Region':'Total'})
sh_Region.index.name = 'Region'

In [7]:
# Synthetic Persons (SP)
sp_mgra = pd.DataFrame(synthetic_persons['mgra'].value_counts()).rename(columns={'mgra':'Total'})
sp_mgra.index.name = 'mgra'

sp_tract = pd.DataFrame(synthetic_persons['tract'].value_counts()).rename(columns={'tract':'Total'})
sp_tract.index.name = 'tract'

sp_CPA = pd.DataFrame(synthetic_persons['CPA'].value_counts()).rename(columns={'CPA':'Total'})
sp_CPA.index.name = 'CPA'

sp_jurisdiction = pd.DataFrame(synthetic_persons['Jurisdiction'].value_counts()).rename(columns={'Jurisdiction':'Total'})
sp_jurisdiction.index.name = 'Jurisdiction'

sp_LUZ = pd.DataFrame(synthetic_persons['LUZ'].value_counts()).rename(columns={'LUZ':'Total'})
sp_LUZ.index.name = 'LUZ'

sp_Region = pd.DataFrame(synthetic_persons['Region'].value_counts()).rename(columns={'Region':'Total'})
sp_Region.index.name = 'Region'

# Internal Consistency Checks
Reminder: Tract and CPA should not match 

In [8]:
# Synthetic Persons
print(f"MGRA total - {sp_mgra['Total'].sum()}")
print(f"Tract total - {sp_tract['Total'].sum()}")
print(f"CPA total - {sp_CPA['Total'].sum()}")
print(f"Jurisdiction Total - {sp_jurisdiction['Total'].sum()}")
print(f"LUZ total - {sp_LUZ['Total'].sum()}")
print(f"Region total - {sp_Region['Total'].sum()}")

MGRA total - 3361280
Tract total - 3247180
CPA total - 1440291
Jurisdiction Total - 3361280
LUZ total - 3361280
Region total - 3361280


In [9]:
# Synthetic Households
print(f"MGRA total - {sh_mgra['Total'].sum()}")
print(f"Tract total - {sh_tract['Total'].sum()}")
print(f"CPA total - {sh_CPA['Total'].sum()}")
print(f"Jurisdiction Total - {sh_jurisdiction['Total'].sum()}")
print(f"LUZ total - {sh_LUZ['Total'].sum()}")
print(f"Region total - {sh_Region['Total'].sum()}")

MGRA total - 1267147
Tract total - 1153047
CPA total - 579807
Jurisdiction Total - 1267147
LUZ total - 1267147
Region total - 1267147


# Comparison with Forecasts
- Can't do a one to one check, will need to check a rough estimate
- The only forecast available at the moment is the 2019 base year forecast 

In [12]:
forecast_data_mgra = pd.read_csv('J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_15_2019_CSV_outputs/mgra_2019_CSV_Data_ind_QA.csv')

## Comparing Population Values at MGRA/Juridiction/Region Levels

### MGRA Level Check

In [30]:
mgra_population = forecast_data_mgra[['mgra','pop']]
mgra_persons_comparison = sp_mgra.reset_index().merge(mgra_population, how='left', on='mgra')
mgra_persons_comparison.columns = ['mgra', 'popsim_pop', 'forecast_pop']
mgra_persons_comparison['Diff'] = mgra_persons_comparison['popsim_pop'] - mgra_persons_comparison['forecast_pop']
mgra_persons_comparison = mgra_persons_comparison.sort_values('Diff', ascending=False)

mgra_persons_comparison

,mgra,popsim_pop,forecast_pop,Diff
2,4929,8528,4264,4264
9,7507,4228,3623,605
15,19747,3194,2742,452
0,5878,11396,11018,378
14,2428,3395,3028,367
...,...,...,...,...
59,19499,1465,1610,-145
592,2768,673,821,-148
30,6963,1952,2105,-153
32,13437,1852,2013,-161


# Comparison with LEHD Data